## 什么是Agents ?

Agents 是一个具有智能功能的智能体，它使用 LLM 和工具来执行任务。

Agents 核心思想是使用LLM来选择要采取的一系列动作。在链式结构中，一系列动作是硬编码的（在代码中）。 在 Agents 中，使用语言模型作为推理引擎来确定要采取的动作及其顺序。

Agents 包括几个关键组件：

- **Agent**: 用于生成指令和执行动作的代理。
- **Tool**: 用于执行动作的函数。
- **Memory**: 用于存储历史对话和生成的指令。
- **LLM**: 用于生成指令和执行动作的 LLM。

Agents 是什么？

重新定义一下




## 第一个例子

- 会做数学题
- 不知道答案的时候可以搜索

搭建工具
- serpai是一个聚合搜索引擎，需要安装谷歌搜索包以及申请账号 https://serpapi.com/manage-api-key
- llm-math是一个封装好的数学计算链

pip install google-search-results

In [2]:
import os 
os.environ["SERPAPI_API_KEY"] = 'db166b810c6b85674b6ceab3bd4e10d5048e1ba837db1c0d962ad91b34558805'

In [3]:
from langchain_community.llms.tongyi import Tongyi

llm = Tongyi()

In [4]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(["serpapi","llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,#这里有不同的类型
    verbose=True,#是否打印日志
)

/opt/anaconda3/envs/py311/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [5]:
llm.invoke("请问2024年的美国总统是谁？他的年龄的除以2是多少?")

'目前无法预测2024年的美国总统会是谁，因此也无法计算其年龄及其一半。当前的美国总统是乔·拜登（Joe Biden），但他是否会在2024年再次当选或由其他候选人胜出，还有待观察和选举结果来确定。一旦确定了总统人选及其出生年份，您可以通过简单的数学计算得出其年龄及其一半。'

In [6]:
agent.run("请问现任的美国总统是谁？他的年龄的除以2是多少?")

/opt/anaconda3/envs/py311/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


我需要先找出现任的美国总统是谁，然后找到他的年龄，最后将他的年龄除以2。
Action: Search
Action Input: 现任美国总统

Observation: Joe Biden
Thought:现在我知道了现任美国总统是乔·拜登，接下来我需要查找他的年龄。
Action: Search
Action Input: Joe Biden age


Observation: 81 years
Thought:现在我知道了乔·拜登的年龄是81岁，最后一步是用计算器将他的年龄除以2。
Action: Calculator
Action Input: 81 / 2

Observation: Answer: 40.5
Thought:我现在知道了最终答案。
Final Answer: 乔·拜登的年龄除以2是40.5。

> Finished chain.


'乔·拜登的年龄除以2是40.5。'

## Agents 的类型

- ZERO_SHOT_REACT_DESCRIPTION                   零样本反应描述
- CHAT_ZERO_SHOT_REACT_DESCRIPTION              聊天零样本反应描述
- CONVERSATIONAL_REACT_DESCRIPTION              会话反应描述
- CHAT_CONVERSATIONAL_REACT_DESCRIPTION         聊天会话反应描述
- STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION   聊天结构化零样本反应描述
- STRUCTURED_ZERO_SHOT_REACT_DESCRIPTION        结构化零样本反应描述

### ZERO_SHOT_REACT_DESCRIPTION

即在没有示例的情况下可以自主的进行对话的类型。
应用场景：
主要用于即时响应描述性任务，这些任务可能不需要复杂的对话上下文，而是直接针对某个问题或请求给出描述性回答。
适用于那些需要快速生成准确描述的场景，如产品特性说明、数据报告解读等。
交互方式：
通常以单轮问答的形式进行，用户输入一个问题或请求，系统直接返回相应的描述性回答。
不涉及复杂的对话历史或会话管理。
生成内容特点：
生成的回答侧重于直接、准确的描述，可能不包含过多的对话性语言或上下文依赖。

In [7]:
from langchain.agents import load_tools
# 定义tools
tools = load_tools(["serpapi","llm-math"],llm=llm)

# 定义agent--（tools、agent、llm、memory）
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# print(agent)
# print("------------------------")
# print(agent.agent.llm_chain.prompt.template)

agent.invoke("现在美国总统是谁？他的年龄除以2是多少？")




> Entering new AgentExecutor chain...


I need to find out who the current President of the United States is and then calculate their age divided by 2.
Action: Search
Action Input: current President of the United States

Observation: Joe Biden
Thought:I need to find out Joe Biden's age and then divide it by 2.
Action: Search
Action Input: Joe Biden age


Observation: 81 years
Thought:I need to divide Joe Biden's age by 2.
Action: Calculator
Action Input: 81 / 2


KeyboardInterrupt: 

### CHAT_ZERO_SHOT_REACT_DESCRIPTION

零样本增强式生成，即在没有示例的情况下可以自主的进行对话的类型。
应用场景：
专为聊天和对话场景设计，能够处理多轮对话，并在对话过程中生成连贯、自然的回答。
适用于需要与用户进行交互、理解用户意图并给出相应反应的场景，如聊天机器人、客服系统等。
交互方式：
支持多轮对话，能够处理用户的连续输入，并根据对话历史生成更加准确的回答。
可能需要管理对话状态、跟踪用户意图和上下文信息。
生成内容特点：
生成的回答更加自然、流畅，能够适应用户的语言风格和对话习惯。
可能包含对话性语言、问候语、确认信息等，以增强用户体验。

In [28]:
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
# print(agent)
# print("------------------------")
# print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
# print("------------------------")
agent.invoke("现在美国总统是谁？他的年龄除以2是多少？")




> Entering new AgentExecutor chain...
Thought: 我需要先找出当前美国总统的信息，然后计算他的年龄除以2。
Action:
```
{
  "action": "Search",
  "action_input": "current President of the United States"
}
```
Observation: Joe Biden
Thought:现在我知道了美国总统是乔·拜登。我需要查找他的年龄，然后用计算器来完成除法运算。
Action:
```
{
  "action": "Search",
  "action_input": "age of Joe Biden"
}
```

Observation: 81 years
Thought:我现在知道了乔·拜登的年龄是81岁，接下来我需要用计算器来计算81除以2。
Action:
```
{
  "action": "Calculator",
  "action_input": "81 / 2"
}
```

Observation: Answer: 40.5
Thought:我现在知道乔·拜登的年龄除以2的结果是40.5。
Final Answer: 现在美国总统是乔·拜登，他的年龄除以2是40.5。

> Finished chain.


{'input': '现在美国总统是谁？他的年龄除以2是多少？', 'output': '现在美国总统是乔·拜登，他的年龄除以2是40.5。'}

### CONVERSATIONAL_REACT_DESCRIPTION 

一个对话型的agent，这个agent要求与memory一起使用

In [45]:
from langchain.memory import ConversationBufferMemory
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
)

# 定义tool
tools = load_tools(["serpapi","llm-math"],llm=llm)

# 定义agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.template)

memory=ConversationBufferMemory(memory_key='chat_history') verbose=True tags=['conversational-react-description'] agent=ConversationalAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Ass

In [46]:
agent.run("我是张三，今年18岁，性别女，现在在深圳工作，工作年限1年，月薪5000元")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: 你好，张三！很高兴认识你。你在深圳的工作经验听起来很有趣。你能告诉我更多关于你的工作吗？

> Finished chain.


'你好，张三！很高兴认识你。你在深圳的工作经验听起来很有趣。你能告诉我更多关于你的工作吗？'

In [13]:
agent.run("我的名字是什么?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: 你之前提到你的名字是张三。是不是有什么混淆了呢？

> Finished chain.


'你之前提到你的名字是张三。是不是有什么混淆了呢？'

In [ ]:
agent.run("有什么好吃的泰国菜可以推荐给我吗?")

In [ ]:
agent.run("这些我都没吃过！我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？")

In [14]:
agent.run("中国陕西西安现在的气温多少？截止目前我们聊了什么？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: 中国陕西西安现在的气温

Observation: {'type': 'weather_result', 'temperature': '80', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '82%', 'wind': '1 mph', 'location': '中国陕西省西安市', 'date': 'Wednesday 9:00 AM', 'weather': 'Partly cloudy'}
Thought:The current temperature in Xi'an, Shaanxi, China is 80 degrees Fahrenheit with partly cloudy weather. We have talked about your name, which is Zhang San, your age, gender, location, work experience, and your salary. You asked me to clarify your name and then inquired about the current temperature in Xi'an.
Thought: Do I need to use a tool? No
AI: 目前西安的气温是80华氏度，多云。在此之前，我们聊了关于你的基本信息，包括你的名字张三、年龄、性别、所在城市、工作经验以及月薪。有什么其他问题或信息你想了解的吗？

> Finished chain.


'目前西安的气温是80华氏度，多云。在此之前，我们聊了关于你的基本信息，包括你的名字张三、年龄、性别、所在城市、工作经验以及月薪。有什么其他问题或信息你想了解的吗？'

### CHAT_CONVERSATIONAL_REACT_DESCRIPTION 使用了chatmodel
代理是 LangChain 框架中针对聊天和会话场景设计的一种高级代理类型。它利用 React 框架（在 LangChain 的上下文中，React 框架指的是一种用于决定何时调用哪个工具的逻辑框架，而非前端开发中的 React 库）来动态地选择和执行工具，同时利用会话记忆（如对话历史）来增强交互的连贯性和上下文理解能力。

In [49]:
from langchain.memory import ConversationBufferMemory
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

tools = load_tools(["serpapi","llm-math"],llm=llm)


agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    verbose=True,
)
print(agent)
print("1 ------------------------")
print(len(agent.agent.llm_chain.prompt.messages))
print("2 ------------------------")
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
print("3 ------------------------")
print(agent.agent.llm_chain.prompt.messages[1])
print("4 ------------------------")
print(agent.agent.llm_chain.prompt.messages[2].prompt.template)
print("5 ------------------------")
print(agent.agent.llm_chain.prompt.messages[3])



memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['chat-conversational-react-description'] agent=ConversationalChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Ass

In [50]:
agent.run("有什么好吃的泰国菜可以推荐给我吗?用中文回答")



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "泰国菜 推荐 中文"
}
```
Observation: ['看起来简简单单的炒饭，味道却完全不普通哦！跟我们熟悉的中式炒饭不同，泰式炒饭首先使用的都是香米，这就让炒饭香气更浓，而且更加粒粒分明。同时，炒饭 ...', '泰國10 大美食 · 曼谷當地必吃美食 · 辣蝦湯 · 辣味青木瓜沙拉 · 椰子燉雞湯 · 紅咖哩 · 泰式炒河粉 · 泰式炒飯 · 打拋豬肉.', '值得推荐的菜式有鸡蛋面加叉烧（Bamee Keaw Daeng），叉烧加米饭（Khao Moo Daeng），白切鸡（Khao Man Kai），牛肉面，烧烤海鲜，泰式寿喜烧，青木瓜沙拉和烤肉。 东 ...', '毋庸置疑，泰国菜美味可口，举世闻名。 · 1. 酸汤虾ต้มยำกุ้ง (冬阴功) · 2. 绿咖喱鸡肉แกงเขียวหวาน (Geng Kiaw Waen Gai) · 3. 炒面ผัดไทย (Pad Tai) · 4. Pork Ba\u200b\u200bked in Basil ...', '咖哩汤河（khao soi; 泰文ข้าวซอย) )。 当地还有一种特别的酸肉，叫做“nem”。 泰东北菜，和老挝菜相似，而米饭则爱吃糯米饭。 生肉沙拉（音：larb; 泰文： ...', '受欢迎的泰国美食，如冬阴功汤、泰式炒河粉、绿咖喱和Som Tam，只是构成这个充满活力的国家美食的众多美味佳肴中的一部分。 无论您是想吃辣的还是甜的——受 ...', '米其林评审员的评语：对我来说，这是泰国最好的鱼肉冬阴汤。厨师找到了很好的味道平衡：酸、咸、辣和最后的一点点甜味。所有的味道都与鱼肉都搭配 ...', '泰国菜有四大菜系，分别为泰北菜、泰东北菜、泰中菜、与泰南菜，反映泰国四方不同的地理和文化，而各地使用的食材往往跟邻近国家的一样。 泰国南部菜.', '值得推荐的菜式有鸡蛋面加叉烧（Bamee Keaw Daeng），叉烧加米饭（Khao Moo Daeng），白切鸡（Khao Man Kai），牛肉面，烧烤海鲜，泰式寿喜烧，青木瓜沙拉和烤肉。 东 ...', '这一期的5280杂志推荐了5家

'根据查找的信息，一些值得推荐的泰国菜包括：泰式炒饭、泰式炒河粉、冬阴功汤、绿咖喱鸡肉、青木瓜沙拉、椰子炖鸡汤、红咖喱、鸡蛋面加叉烧、叉烧配米饭、白切鸡、牛肉面、烧烤海鲜、泰式寿喜烧等。这些菜肴各具特色，你可以根据自己的口味选择尝试。'

### STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION 

对输出做了结构化处理

In [62]:

from langchain.memory import ConversationBufferMemory
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

## 定义tool
tools = load_tools(["serpapi","llm-math"],llm=llm)

# 定义agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, #agent类型 
    memory=memory,#记忆组件
    handle_parsing_errors=True,
    verbose=True,
    
)
# print(agent)
# print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
# print(agent.agent.llm_chain.prompt.messages[1].prompt.template)

In [63]:
agent.run("有什么好吃的泰国菜可以推荐给我吗?用中文回答")



> Entering new AgentExecutor chain...
{
  "action": "Search",
  "action_input": "泰国菜 推荐"
}
```



> Finished chain.


'{\n  "action": "Search",\n  "action_input": "泰国菜 推荐"\n}\n```\n\n'

## Tools

langchain预制了大量的tools，基本这些工具能满足大部分需求。 https://python.langchain.com.cn/docs/modules/agents/tools/

- 加载预制tool的方法
- 几种tool的使用方式

In [18]:
#添加预制工具的方法很简单
from langchain.agents import load_tools
tool_names = [...]
tools = load_tools(tool_names) #使用load方法

#有些tool需要单独设置llm
from langchain.agents import load_tools
tool_names = [...]
llm = ...
tools = load_tools(tool_names, llm=llm) #在load的时候指定llm

ValueError: Got unknown tool Ellipsis

### SerpAPI

最常见的聚合搜索引擎 https://serper.dev/dashboard，支持google\bing

In [19]:
from langchain.utilities.serpapi import SerpAPIWrapper

# 实例化
search = SerpAPIWrapper()

search.run("美国现在的总统是谁？")


"['1951年开始生效的美国宪法第二十二修正案规定总统每届任期4年，连选连任不得多于2次，也不能担任总统或执行总统职责超过2年后再当选为总统多于1次。作为当今世界上唯一的 ...', '拜登曾在1988年和2008年竞选过民主党的总统候选人提名，但都没有成功。2019年4月，拜登宣布参加隔年的总统大选，并成功获得党内总统候选人提名。2020年11月，拜 ...', '美国前总统唐纳德·特朗普（Donal Trump，又译川普）在爱荷华州的共和党初选“党团会议”（ Caucus ）投票中大胜，这也巩固了他在2024年共和党初选明确的 ...', '美国总统（President of the United States of America，缩写POTUS）是美利坚合众国的国家元首、政府首脑与三军统帅，有时被称为Mr. President、The Honorable ...', '这位民主党人在与共和党对手、78 岁的唐纳德·特朗普的辩论中表现不佳，承受了数周的压力，于周日宣布决定辞职。 这位共和党候选人在北卡罗来纳州夏洛特对 ...', '新华社华盛顿3月12日电（记者熊茂伶胡友松）美国现任总统拜登和前总统特朗普12日各自锁定2024年总统选举民主党总统候选人提名和共和党总统候选人提名 ...', '特朗普与拜登很有可能将再次在总统大选中对垒，对美国来说，这将超越自由与保守的两党二元对决，而更是关乎种族、文化、经济以及美国人认同感的决战。', '特朗普总统就是这样。他指责有选举欺诈，并表示要继续挑战选举结果。他的立场使美国国会议员处于分裂状态。共和党人支持对他们 ...', '通用英语。根据2023年盖洛普民调数据，美国人口中约68％信仰基督教，其中33％信仰新教、22％信仰天主教、 ...', '乔·拜登将于2021年1月20日宣誓就任美国总统。他是怎样一个人？从政生涯有哪些亮点？白宫之路遭遇哪些坎坷？人生中又有过哪些喜怒哀乐？']"

In [37]:
# 支持自定义参数，比如将引擎切换到bing，设置搜索语言等
params = {
    "engine": "bing",
    "gl": "us",
    "hl": "en",
}
search = SerpAPIWrapper(params=params)
search.run("美国现在的总统是谁？")

"{'type': 'fact', 'breadcrumb': 'United States / President'}"

In [20]:
tools = load_tools(["serpapi"],llm=llm)

### Dall-E

Dall-E是openai出品的文到图AI大模型

pip install opencv-python scikit-image



In [8]:
from langchain.agents import initialize_agent, load_tools

tools = load_tools(["dalle-image-generator"])

agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description",
    verbose=True
)

agent.run("Create an image of a halloween night at a haunted museum")

ValidationError: 1 validation error for DallEAPIWrapper
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

### Eleven Labs Text2Speech
ElevenLabs 是非常优秀的TTS合成API

```
pip install elevenlabs
pip install --upgrade pydantic
```

In [9]:
import os

os.environ["ELEVEN_API_KEY"] = "23261e4a3b79697822252a505a169863"
from langchain.tools.eleven_labs import ElevenLabsText2SpeechTool

text_to_speak = "Hello! 你好! Hola! नमस्ते! Bonjour! こんにちは! مرحبا! 안녕하세요! Ciao! Cześć! Привіт! வணக்கம்!"

tts = ElevenLabsText2SpeechTool(
    voice="Bella",
    text_to_speak=text_to_speak,
    verbose=True
)
tts.name

'eleven_labs_text2speech'

In [ ]:
speech_file = tts.run(text_to_speak)

speech_file = tts.run(text_to_speak)

tts.stream_speech(text_to_speak)

In [ ]:
from langchain.agents import initialize_agent, load_tools

tools = load_tools(["eleven_labs_text2speech"])

agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description",
    verbose=True
)

agent.run("Create an image of a halloween night at a haunted museum")

### GraphQL
一种api查询语言，类似sql，我们用它来查询奈飞的数据库，查找一下和星球大战相关的电影，API地址https://swapi-graphql.netlify.app/.netlify/functions/index

```
pip install httpx gql > /dev/null
pip install gql
pip install requests_toolbelt
```

In [25]:
### from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.utilities import GraphQLAPIWrapper


tools = load_tools(
    ["graphql"],
    graphql_endpoint="https://swapi-graphql.netlify.app/.netlify/functions/index",
)

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)



ImportError: Could not import gql python package. Try installing it with `pip install gql`. Received error: No module named 'gql'

# Tookit
tookit是langchain已经封装好的一系列工具，一个工具包是一组工具来组合完成特定的任务


## python
一个python代码机器人

 pip install langchain_experimental==0.0.4

In [14]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool
from langchain_experimental.utilities import PythonREPL
from langchain.agents.agent_types import AgentType

agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    agent_executor_kwargs={"handle_parsing_errors": True},
)

agent_executor.run("生成10个斐波那契数列?")

ModuleNotFoundError: No module named 'langchain_experimental'

In [56]:
def generate_fibonacci_sequence(n):
    sequence = []
    a, b = 0, 1
    for _ in range(n):
        sequence.append(a)
        a, b = b, a + b
    return sequence

fibonacci_sequence = generate_fibonacci_sequence(10)
fibonacci_sequence

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

## SQL Database
使用SQLDatabaseChain构建的agent，用来根据数据库回答一般行动饿问题

In [41]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain_community.chat_models import ChatTongyi

In [48]:
db = SQLDatabase.from_uri("sqlite:///db/Chinook.db")

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [51]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [57]:
agent_executor.run("创建用户表，包括用户名和密码两个列")



> Entering new AgentExecutor chain...
```python
# Assuming we're using SQLite for this demonstration
import sqlite3

# Connect to a new database (or an existing one)
connection = sqlite3.connect('users.db')
cursor = connection.cursor()

# Create the 'users' table with 'username' and 'password' columns
cursor.execute('''CREATE TABLE IF NOT EXISTS users
                 (username TEXT PRIMARY KEY, password TEXT)''')

# Commit the changes and close the connection
connection.commit()
connection.close()
```
This code creates a SQLite database named 'users.db' and a table called 'users' with two columns: 'username' (set as the primary key) and 'password'. Both columns are of type TEXT.

> Finished chain.


"```python\n# Assuming we're using SQLite for this demonstration\nimport sqlite3\n\n# Connect to a new database (or an existing one)\nconnection = sqlite3.connect('users.db')\ncursor = connection.cursor()\n\n# Create the 'users' table with 'username' and 'password' columns\ncursor.execute('''CREATE TABLE IF NOT EXISTS users\n                 (username TEXT PRIMARY KEY, password TEXT)''')\n\n# Commit the changes and close the connection\nconnection.commit()\nconnection.close()\n```\nThis code creates a SQLite database named 'users.db' and a table called 'users' with two columns: 'username' (set as the primary key) and 'password'. Both columns are of type TEXT."

In [15]:
agent_executor.run("""sqlite数据库中的users表的表结构如下：\n
                   CREATE TABLE IF NOT EXISTS users\n
                 (username TEXT PRIMARY KEY, password TEXT)\n
                 向uesrs表中插入两条数据""")

NameError: name 'agent_executor' is not defined

# 自定义Agent

- 定义一个class
- 工具：默认搜索
- 提示词：定义agent要做什么任务
- outparse：约束LLM的行为和输出
- 不同的LLM不同的质量

## 自定义Tool

In [33]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain_community.llms.tongyi import Tongyi
import re
import os

class MyAgentTool:
    
    def __init__(self) -> None:    
        os.environ["SERPAPI_API_KEY"] = 'db166b810c6b85674b6ceab3bd4e10d5048e1ba837db1c0d962ad91b34558805'    
        self.serpapi = SerpAPIWrapper()
        
    def tools(self):
        return [
            Tool(
                name="search",
                description="适用于当你需要回答关于当前事件的问题时",
                func=self.serpapi.run,
            )
        ]
s = MyAgentTool()
s.serpapi.run("python")
    

"['Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured, object-oriented and functional programming.', 'Python type: High-level programming language.', 'Python entity_type: kp3_verticals.', 'Python kgmid: /m/05z1_.', 'Python designed_by: Guido van Rossum.', 'Python developer: Python Software Foundation.', 'Python filename_extensions: .py,.pyw,.pyz,.pyi,.pyc,.pyd.', 'Python first_appeared: 20 February 1991; 33 years ago.', 'Python paradigm: Multi-paradigm - object-oriented, procedural (imperative), functional, structured, reflective.', 'Python stable_release: 3.12.4 / 6 June 2024; 53 days ago.', 'Python typing_discipline: duck, dynamic, strong; optional type annotations (since 3.5, but those hints are ignored, except with unofficial tools).', 'The official home of the Python

## 自定义agents

In [17]:
from typing import Any


class MyAgent:
    llm = None
    tools = None
    def __init__(self) -> None:
        #agent的提示词，用来描述agent的功能
        self.template =  """尽你最大可能回答下面问题，你将始终用中文回答. 你在必要时可以使用下面这些工具:
                    {tools}
                    Use the following format:
                    Question: the input question you must answer
                    Thought: you should always think about what to do
                    Action: the action to take, should be one of [{tool_names}]
                    Action Input: the input to the action
                    Observation: the result of the action
                    ... (this Thought/Action/Action Input/Observation can repeat N times)
                    Thought: I now know the final answer
                    Final Answer: the final answer to the original input question
                    Begin! 记住使用中文回答，如果你使用英文回答将回遭到惩罚.
                    Question: {input}
                    {agent_scratchpad}"""
        #定义一个openai的llm
        self.llm = Tongyi()
        #工具列表
        self.tools = self.MyAgentTool().tools()
        #agent的prompt
        self.prompt = self.MyTemplate(
            template=self.template,
            tools=self.tools,
            #输入变量和中间变量
            input_variables=["input", "intermediate_steps"],
        )

        #定义一个LLMChain
        self.llm_chain = LLMChain(
            llm=self.llm,
            prompt = self.prompt
        )
        #工具名称列表
        self.toolnames = [tool.name for tool in self.tools]
        #定义一个agent
        self.agent = LLMSingleActionAgent(
            llm_chain=self.llm_chain,
            allowed_tools=self.toolnames,
            output_parser=self.MyOutputParser(),
            
            stop=["\nObservation:"],
        )
    
    #运行agent
    def run(self, input: str) -> str:
        #创建一个agent执行器
        agent_executor = AgentExecutor.from_agent_and_tools(
            agent=self.agent, 
            tools=self.tools, 
            handle_parsing_errors=True,
            verbose=True
        )
        agent_executor.run(input=input)

    #自定义工具类
    class MyAgentTool:
        def __init__(self) -> None:
            os.environ["SERPAPI_API_KEY"] = 'db166b810c6b85674b6ceab3bd4e10d5048e1ba837db1c0d962ad91b34558805'    
            self.serpapi = SerpAPIWrapper()
            
        def tools(self):
            return [
                Tool(
                    name="search",
                    description="适用于当你需要回答关于当前事件的问题时",
                    func=self.serpapi.run,
                )
            ]
    

    #自定义模版渲染类
    class MyTemplate(StringPromptTemplate):
        #渲染模版
        template: str
        #需要用到的工具
        tools:List[Tool]

        #格式化函数
        def format(self, **kwargs: Any) -> str:
            #获取中间步骤
            intermediate_steps = kwargs.pop("intermediate_steps")
            thoughts = ""
            for action, observation in intermediate_steps:
                thoughts += action.log
                thoughts += f"\nObservation: {observation}\nThought: "
            #将agent_scratchpad设置为该值
            kwargs["agent_scratchpad"] = thoughts
            # 从提供的工具列表中创建一个名为tools的变量
            kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
            #创建一个提供的工具名称列表
            kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
            return self.template.format(**kwargs)


    #自定义输出解析类
    class MyOutputParser(AgentOutputParser):
        #解析函数
        def parse(self, output: str) -> Union[AgentAction, AgentFinish]:
            #检查agent是否应该完成
            if "Final Answer:" in output:
                return AgentFinish(
                # 返回值通常始终是一个具有单个 `output` 键的字典。
                # It is not recommended to try anything else at the moment :)
                return_values={"output": output.split("Final Answer:")[-1].strip()},
                log=output,
                )
            #用正则解析出动作和动作输入
            regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
            match = re.search(regex, output, re.DOTALL)
            #如果没有匹配到则抛出异常
            if not match:
                raise OutputParserException(f"Could not parse LLM output: `{output}`")
            action = match.group(1).strip()
            action_input = match.group(2)
            # 返回操作和操作输入
            return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=output)

    

In [18]:
myagent = MyAgent()
myagent.run("请问现任的美国总统是谁？他的年龄的除以2是多少?")

/opt/anaconda3/envs/py311/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.agent.LLMSingleActionAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/opt/anaconda3/envs/py311/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: 我需要查找现任美国总统的信息来回答这个问题。
Action: search
Action Input: 现任美国总统


Observation:Joe Biden
我已经知道现任美国总统是乔·拜登，现在我需要查找他的年龄。
Action: search
Action Input: 乔·拜登 年龄



Observation:81 years
我已经找到了乔·拜登的年龄是81岁，现在我需要计算他的年龄除以2。
Action: search
Action Input: 81 divided by 2


Observation:40.5
我现在知道了最终答案。
Final Answer: 现任美国总统是乔·拜登，他的年龄除以2是40.5。

> Finished chain.


## 案例

In [43]:
from langchain.agents import Tool, tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain_community.llms.tongyi import Tongyi
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
# 引入向量化的类
from langchain_community.vectorstores import Chroma
import re
import os

from typing import Any

#自定义工具类
class MyAgentTool:
    def __init__(self) -> None:
        self.db = Chroma(embedding_function=DashScopeEmbeddings(),
                          persist_directory="./chroma")
        
    def tools(self):
        return [
            Tool(
                name="kg_search",
                description="当你需要回答2024年NBA冠军球队的问题时使用",
                func=self.search
            )
        ]
    
    def search(self, query: str) -> str:
        return self.db.similarity_search(query,k=2)
    
    def getdb(self):
        return self.db.__len__()
    
#可用工具
@tool
def kgg_search(query: str): 
    """当你需要回答2024年NBA冠军球队的问题时才会使用这个工具。"""
    db = Chroma(embedding_function=DashScopeEmbeddings(),persist_directory="./chroma")
    return db.similarity_search(query,k=2)

tool_list = [kgg_search]
    

class MyAgent:
    llm = None
    tools = None
    def __init__(self,llm,tools) -> None:
        #agent的提示词，用来描述agent的功能
        self.template =  """尽你最大可能回答下面问题，你将始终用中文回答. 你在必要时可以使用下面这些工具:
                    {tools}
                    使用下面的格式回答问题:
                    问题: 输入的问题
                    分析: 你对问题的分析，决定是否使用工具
                    动作: 使用工具，工具名称 [{tool_names}]
                    输入: {input}
                    观察: 动作名称
                    ... 
                    分析: 我现在知道答案了
                    答案: 这是最终的答案
                    记住使用中文回答，如果你使用英文回答将回遭到惩罚.
                    问题: {input}
                    {agent_scratchpad}"""
                    
        #定义一个openai的llm
        self.llm = llm
        #工具列表
        self.tools = tools
        #agent的prompt
        self.prompt = self.MyTemplate(
            template=self.template,
            tools=self.tools,
            #输入变量和中间变量
            input_variables=["input", "intermediate_steps"],
        )

        #定义一个LLMChain
        self.llm_chain = LLMChain(
            llm=self.llm,
            prompt = self.prompt
        )
        #工具名称列表
        self.toolnames = [tool.name for tool in self.tools]
        #定义一个agent
        self.agent = LLMSingleActionAgent(
            llm_chain=self.llm_chain,
            allowed_tools=self.toolnames,
            output_parser=self.MyOutputParser(),
            
            stop=["\n观察:"],
        )
    
    #运行agent
    def run(self, input: str) -> str:
        #创建一个agent执行器
        agent_executor = AgentExecutor.from_agent_and_tools(
            agent=self.agent, 
            tools=self.tools, 
            handle_parsing_errors=True,
            verbose=True
        )
        # agent_executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=True,handle_parsing_errors=True)
        agent_executor.run(input=input)

    

    #自定义模版渲染类
    class MyTemplate(StringPromptTemplate):
        #渲染模版
        template: str
        #需要用到的工具
        tools:List[Tool]

        #格式化函数
        def format(self, **kwargs: Any) -> str:
            #获取中间步骤
            intermediate_steps = kwargs.pop("intermediate_steps")
            thoughts = ""
            for action, observation in intermediate_steps:
                thoughts += action.log
                thoughts += f"\n观察: {observation}\n想法: "
            #将agent_scratchpad设置为该值
            kwargs["agent_scratchpad"] = thoughts
            # 从提供的工具列表中创建一个名为tools的变量
            kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
            #创建一个提供的工具名称列表
            kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
            prompt_ret = self.template.format(**kwargs)
            return prompt_ret


    #自定义输出解析类
    class MyOutputParser(AgentOutputParser):
        #解析函数
        def parse(self, output: str) -> Union[AgentAction, AgentFinish]:
            #检查agent是否应该完成
            if "答案:" in output:
                return AgentFinish(
                # 返回值通常始终是一个具有单个 `output` 键的字典。
                # It is not recommended to try anything else at the moment :)
                return_values={"output": output.split("答案:")[-1].strip()},
                log=output,
                )
            #用正则解析出动作和动作输入
            regex = r"动作\s*\d*\s*:(.*?)\n输入\s*\d*\s*:[\s]*(.*)"
            match = re.search(regex, output, re.DOTALL)
            #如果没有匹配到则抛出异常
            if not match:
                raise OutputParserException(f"Could not parse LLM output: `{output}`")
            action = match.group(1).strip()
            action_input = match.group(2)
            # 返回操作和操作输入
            return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=output)

myagent = MyAgent(Tongyi(),tool_list)
myagent.run("2024年NBA冠军球队是哪只?")

# tool = MyAgentTool()
# tool.search("2024年NBA冠军球队是哪只")



> Entering new AgentExecutor chain...
分析: 需要使用工具来获取2024年NBA冠军球队的信息。
动作: 使用工具 [kgg_search]
输入: 2024 NBA Champion

观察:使用工具 [kgg_search] is not a valid tool, try one of [kgg_search].
我无法预测未来的事件，比如2024年NBA冠军球队。这需要等到那个时间点通过实际比赛结果来确定。
答案: 目前无法预测2024年NBA冠军球队，这需要等待比赛结果。

> Finished chain.


In [19]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    MyAgentTool().tools(),
    Tongyi(),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
# agent.run("2024年NBA冠军球队是哪只?")
agent.invoke("2024年NBA冠军球队是哪只?")


/opt/anaconda3/envs/py311/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
这个问题是关于未来事件的，我无法通过搜索得到答案，因为现在的时间点上这个事件还没有发生。根据当前时间点的知识，无法确定2024年的NBA冠军。
Thought: 我现在知道最终答案。
Final Answer: 对于2024年NBA冠军的预测目前无法给出确定的答案，因为这取决于未来的比赛结果和球队表现。

> Finished chain.


{'input': '2024年NBA冠军球队是哪只?',
 'output': '对于2024年NBA冠军的预测目前无法给出确定的答案，因为这取决于未来的比赛结果和球队表现。'}